# Use Amazon SageMaker for Parameter-Efficient Fine Tuning of the ESM-2 Protein Language Model

Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
SPDX-License-Identifier: MIT-0

Note: We recommend running this notebook on a **ml.m5.large** instance with the **Data Science 3.0** image.

### What is a Protein?

Proteins are complex molecules that are essential for life. The shape and structure of a protein determines what it can do in the body. Knowing how a protein is folded and how it works helps scientists design drugs that target it. For example, if a protein causes disease, a drug might be made to block its function. The drug needs to fit into the protein like a key in a lock. Understanding the protein's molecular structure reveals where drugs can attach. This knowledge helps drive the discovery of innovative new drugs.

![Proteins are made up of long chains of amino acids](img/protein.png)

### What is a Protein Language Model?

Proteins are made up of linear chains of molecules called amino acids, each with its own chemical structure and properties. If we think of each amino acid in a protein like a word in a sentence, it becomes possible to analyze them using methods originally developed for analyzing human language. Scientists have trained these so-called, "Protein Language Models", or pLMs, on millions of protein sequences from thousands of organisms. With enough data, these models can begin to capture the underlying evolutionary relationships between different amino acid sequences.

It can take a lot of time and compute to train a pLM from scratch for a certain task. For example, a team at Tsinghua University [recently described](https://www.biorxiv.org/content/10.1101/2023.07.05.547496v3) training a 100 Billion-parameter pLM on 768 A100 GPUs for 164 days! Fortunately, in many cases we can save time and resources by adapting an existing pLM to our needs. This technique is called "fine-tuning" and also allows us to borrow advanced tools from other types of language modeling

### What is LoRA?

One such method originally developed in 2021 for language analysis is ["Low-Rank Adaptation of Large Language Models"](https://www.biorxiv.org/content/10.1101/2023.07.05.547496v3), or "LoRA". This method adapts large pre-trained language models to new tasks. It does this by changing only a small part of the model. This makes the method very efficient. The small changed part targets the most important information needed for the new task. This allows quick customization of the model for new uses.

`peft` is an open source library from HuggingFace to easily run parameter-efficient fine tuning jobs. That includes the use of LoRA. In addition, we'll use int-8 quantization to further increase efficiency.
LoRA + quantization enables us to use less GPU memory (VRAM) to train large language models, giving us more compute flexibility.

### What is ESM-2?

[ESM-2](https://www.biorxiv.org/content/10.1101/2022.07.20.500902v1) is a pLM trained using unsupervied masked language modelling on 250 Million protein sequences by researchers at [Facebook AI Research (FAIR)](https://www.biorxiv.org/content/10.1101/2022.07.20.500902v1). It is available in several sizes, ranging from 8 Million to 15 Billion parameters. The smaller models are suitable for various sequence and token classification tasks. The FAIR team also adapted the 3 Billion parameter version into the ESMFold protein structure prediction algorithm. They have since used ESMFold to predict the struture of [more than 700 Million metagenomic proteins](https://esmatlas.com/about). 

ESM-2 is a powerful pLM. However, it has traditionally required multiple A100 GPU chips to fine-tune. In this notebook, we demonstrate how to use QLoRA to fine-tune ESM-2 in on an inexpensive Amazon SageMaker training instance. We will use ESM-2 to predict [subcellular localization](https://academic.oup.com/nar/article/50/W1/W228/6576357). Understanding where proteins appear in cells can help us understand their role in disease and find new drug targets. 

---
## 1. Set up environment

In [ ]:
%pip install -q -U --disable-pip-version-check -r notebook-requirements.txt

Load the sagemaker package and create some service clients

In [ ]:
import boto3
from datasets import Dataset, DatasetDict
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sagemaker
from sagemaker.experiments.run import Run
from sagemaker.huggingface import HuggingFace, HuggingFaceModel
from sagemaker.inputs import TrainingInput
from time import strftime
from transformers import AutoTokenizer

In [ ]:
boto_session = boto3.session.Session()
sagemaker_session = sagemaker.session.Session(boto_session)
S3_BUCKET = sagemaker_session.default_bucket()
s3 = boto_session.client("s3")
sagemaker_client = boto_session.client("sagemaker")
sagemaker_execution_role = sagemaker.session.get_execution_role(sagemaker_session)
REGION_NAME = sagemaker_session.boto_region_name
print(f"Assumed SageMaker role is {sagemaker_execution_role}")

S3_PREFIX = "esm-loc-ft"
S3_PATH = sagemaker.s3.s3_path_join("s3://", S3_BUCKET, S3_PREFIX)
print(f"S3 path is {S3_PATH}")

EXPERIMENT_NAME = "esm-loc-ft-" + strftime("%Y-%m-%d-%H-%M-%S")
print(f"Experiment name is {EXPERIMENT_NAME}")

---
## 2. Process Data

We'll use a version of the [DeepLoc-2 data set](https://services.healthtech.dtu.dk/services/DeepLoc-2.0/) to fine tune our localization model. It consists of several thousand protein sequences, each with one or more experimentally-observed location labels. This data was extracted by the DeepLoc team at Technical University of Denmark from the public [UniProt sequence database](https://www.uniprot.org/).

In [ ]:
df = pd.read_csv(
    "https://services.healthtech.dtu.dk/services/DeepLoc-2.0/data/Swissprot_Train_Validation_dataset.csv"
).drop(["Unnamed: 0", "Partition"], axis=1)
df["Membrane"] = df["Membrane"].astype("float32")

# filter for sequences between 100 and 512 amino acides
df = df[df["Sequence"].apply(lambda x: len(x)).between(100, 512)]
train = df.sample(frac=0.8)
df = df.drop(train.index)
val = df.sample(frac=0.5)
test = df.drop(val.index)

train = train.reset_index(drop=True)
val = val.reset_index(drop=True)
test = test.reset_index(drop=True)

dataset = DatasetDict(
    {
        "train": Dataset.from_pandas(train, split="train"),
        "test": Dataset.from_pandas(test, split="test"),
        "validation": Dataset.from_pandas(val, split="validation"),
    }
)
print(dataset)

Let's look at the length distribution of our training sequences

In [ ]:
plt.hist(train["Sequence"].apply(lambda x: len(x)).sort_values(), bins=10)

Identify the labels

In [ ]:
labels = [
    label
    for label in dataset["train"].features.keys()
    if label not in ["ACC", "Kingdom", "Membrane", "Sequence"]
]
id2label = {idx: label for idx, label in enumerate(labels)}
label2id = {label: idx for idx, label in enumerate(labels)}
labels

Next, we tokenize the sequences and trim them to a max length of 512 amino acids.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t36_3B_UR50D")


def preprocess_data(examples, max_length=512):
    text = examples["Sequence"]
    encoding = tokenizer(
        text, padding="max_length", truncation=True, max_length=max_length
    )
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    labels_matrix = np.zeros((len(text), len(labels)))
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]
    encoding["labels"] = labels_matrix.tolist()
    return encoding


encoded_dataset = dataset.map(
    preprocess_data,
    batched=True,
    num_proc=os.cpu_count(),
    remove_columns=dataset["train"].column_names,
)
encoded_dataset.set_format("torch")
print(encoded_dataset)

Look at an example record

In [ ]:
example = encoded_dataset["train"][0]
print(tokenizer.decode(example["input_ids"]))
print(example["labels"])
print([id2label[idx] for idx, label in enumerate(example["labels"]) if label == 1.0])

Finally, we upload the processed training, test, and validation data to S3.

In [ ]:
encoded_dataset["train"].save_to_disk(S3_PATH + "/data/train")
encoded_dataset["test"].save_to_disk(S3_PATH + "/data/test")
encoded_dataset["validation"].save_to_disk(S3_PATH + "/data/validation")

If you already have your training and test data prepared in a csv, save the paths in the following cell.

In [ ]:
train_s3_uri = S3_PATH + "/data/train"
test_s3_uri = S3_PATH + "/data/test"

---
## 3. Submit SageMaker Training Jobs

Next, we'll process the 3 Billion-parameter model with LoRA and train on a ml.g5.2xlarge instance.

Define the metrics for SageMaker to extract from the job logs and send to SageMaker Experiments. You can customize these to log more or fewer values.

In [ ]:
metric_definitions = [
    {"Name": "epoch", "Regex": "'epoch': ([0-9.]*)"},
    {"Name": "train_loss", "Regex": "'loss': ([0-9.e-]*)"},
    {"Name": "learning_rate", "Regex": "'learning_rate': ([0-9.e-]*)"},
    {"Name": "train_runtime", "Regex": "'train_runtime': ([0-9.e-]*)"},
    {
        "Name": "train_samples_per_second",
        "Regex": "'train_samples_per_second': ([0-9.e-]*)",
    },
    {
        "Name": "train_steps_per_second",
        "Regex": "'train_steps_per_second': ([0-9.e-]*)",
    },
    {"Name": "eval_loss", "Regex": "'eval_loss': ([0-9.e-]*)"},
    {"Name": "eval_accuracy", "Regex": "'eval_accuracy': ([0-9.e-]*)"},
    {"Name": "eval_f1", "Regex": "'eval_f1': ([0-9.e-]*)"},
    {"Name": "eval_roc_auc", "Regex": "'eval_roc_auc': ([0-9.e-]*)"},
    {"Name": "eval_runtime", "Regex": "'eval_runtime': ([0-9.e-]*)"},
    {
        "Name": "eval_samples_per_second",
        "Regex": "'eval_samples_per_second': ([0-9.e-]*)",
    },
    {"Name": "eval_steps_per_second", "Regex": "'eval_steps_per_second': ([0-9.e-]*)"},
]

### 3.1. Fine-Tune ESM-2 3B on ml.g5.2xlarge using LoRA

In [ ]:
# Additional training parameters
hyperparameters = {
    "epochs": 2,
    "max_length": 512,
    "model_id": "facebook/esm2_t36_3B_UR50D",
    "num_labels": 10,
    "problem_type": "multi_label_classification",
    "per_device_train_batch_size": 2,
    "per_device_eval_batch_size": 32,
    "gradient_accumulation_steps": 16,
}

# creates Hugging Face estimator
huggingface_estimator_g5_3b = HuggingFace(
    base_job_name="esm-3B-loc-ft-lora",
    entry_point="lora-train.py",
    source_dir="scripts/training/peft",
    instance_type="ml.g5.2xlarge",
    instance_count=1,
    transformers_version="4.28",
    pytorch_version="2.0",
    py_version="py310",
    output_path=f"{S3_PATH}/output",
    role=sagemaker_execution_role,
    hyperparameters=hyperparameters,
    metric_definitions=metric_definitions,
    checkpoint_local_path="/opt/ml/checkpoints",
    sagemaker_session=sagemaker_session,
    tags=[{"Key": "project", "Value": "esm-ft"}],
)

In [ ]:
with Run(
    experiment_name=EXPERIMENT_NAME,
    sagemaker_session=sagemaker_session,
) as run:
    huggingface_estimator_g5_3b.fit(
        {
            "train": TrainingInput(s3_data=train_s3_uri, input_mode="File"),
            "test": TrainingInput(s3_data=test_s3_uri, input_mode="File"),
        },
        wait=False,
    )

This training job will take 2 to 3 hours to finish on a ml.g5.2xlarge instance type.

You can view metrics and debugging information for both of these runs in SageMaker Experiments. On the left-side navigation panel, select the Home icon, then "Experiments". From there, you can select your experiment name and training job name and view the Debugger insights.

### 3.2. (Optional) Fine-Tune ESM-2 3B on ml.p4d.24xlarge without LoRA

Uncomment the next cell to fine-tune ESM-2 3B without LoRA. Because of the model size, this requires a ml.p4d.24xlarge instance type with 8xA100 GPUs. To speed up our training, we can use data-distributed parallel (DDP) training to share the samples across all GPU chips.

In [ ]:
# # Additional training parameters
# hyperparameters = {
#     "epochs": 2,
#     "max_length": 512,
#     "model_id": "facebook/esm2_t36_3B_UR50D",
#     "num_labels": 10,
#     "problem_type": "multi_label_classification",
#     "per_device_train_batch_size": 32,
#     "optim": "adamw_torch",
#     "per_device_eval_batch_size": 32,
# }

# # creates Hugging Face estimator
# huggingface_estimator_p4d_3b = HuggingFace(
#     base_job_name="esm-3B-p4d-loc-ft-no-lora",
#     entry_point="train.py",
#     source_dir="scripts/training/peft",
#     instance_type="ml.p4d.24xlarge",
#     instance_count=1,
#     transformers_version="4.28.1",
#     pytorch_version="2.0.0",
#     py_version="py310",
#     output_path=f"{S3_PATH}/output",
#     role=sagemaker_execution_role,
#     hyperparameters=hyperparameters,
#     metric_definitions=metric_definitions,
#     checkpoint_local_path="/opt/ml/checkpoints",
#     sagemaker_session=sagemaker_session,
#     tags=[{"Key": "project", "Value": "esm-ft"}],
# )

# with Run(
#     experiment_name=EXPERIMENT_NAME,
#     sagemaker_session=sagemaker_session,
# ) as run:
#     huggingface_estimator_p4d_3b.fit(
#         {
#             "train": TrainingInput(s3_data=train_s3_uri, input_mode="File"),
#             "test": TrainingInput(s3_data=test_s3_uri, input_mode="File"),
#         },
#         wait=False,
#     )

---
## 4. Deploy LoRA Model as Real-Time Inference Endpoint

In [ ]:
%%time

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    model_data=huggingface_estimator_g5_3b.model_uri,
    role=sagemaker_execution_role,
    transformers_version="4.28.1",
    pytorch_version="2.0.0",
    py_version="py310",
    model_server_workers=1,
    env={"HF_TASK": "text-classification"},
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge"
    role=sagemaker_execution_role
)

In [ ]:
test_seq = "MAAAVVLAAGLRAARRAVAATGVRGGQVRGAAGVTDGNEVAKAQQATPGGAAPTIFSRILDKSLPADILYEDQQCLVFRDVAPQAPVHFLVIPKKPIPRISQAEEEDQQLLGHLLLVAKQTAKAEGLGDGYRLVINDGKLGAQSVYHLHIHVLGGRQLQWPPG"
sample = {"inputs": test_seq}
predictor.predict(sample)

In [ ]:
try:
    predictor.delete_endpoint()
except:
    pass